In [1]:
%load_ext autoreload
%autoreload 2

For testing and development of CondGaussianMatrixProductDistribution objects

In [2]:
import torch

from janelia_core.ml.extra_torch_modules import IndSmpConstantRealFcn
from janelia_core.ml.extra_torch_modules import IndSmpConstantBoundedFcn
from janelia_core.ml.torch_distributions import CondGaussianDistribution
from janelia_core.ml.torch_distributions import CondGaussianMatrixProductDistribution
from janelia_core.ml.torch_distributions import CondMatrixProductDistribution

## Parameters go here

In [3]:
# Matrix shape
n = 10 # No. of rows
m = 3 # No. of cols

## Generate some properties here

In [4]:
 x = torch.rand([n,1])

## Setup the CondGaussianMatrixProduct distributions here 

In [5]:
col_dists_0 = [CondGaussianDistribution(mn_f=IndSmpConstantRealFcn(n=n, init_mn = float(m_i)),
                                  std_f=IndSmpConstantBoundedFcn(n=n))
         for m_i in range(m)]

mat_dist_0 = CondGaussianMatrixProductDistribution(dists=col_dists_0)

In [6]:
col_dists_1 = [CondGaussianDistribution(mn_f=IndSmpConstantRealFcn(n=n, init_mn = float(m_i)),
                                  std_f=IndSmpConstantBoundedFcn(n=n))
         for m_i in range(m)]

mat_dist_1 = CondGaussianMatrixProductDistribution(dists=col_dists_1)

In [7]:
for d in mat_dist_1.dists:
    d.std_f.f.v.data = torch.rand(10) + .5

## Calculate KL divergence between distribitions 0 and 1

In [8]:
mat_dist_0.kl(mat_dist_1, x)

tensor([5.9728, 6.0131, 6.7857, 6.9815, 6.4792, 6.2577, 6.9412, 6.6398, 6.4518,
        5.9142], grad_fn=<AsStridedBackward>)

## Now setup a CondGaussianDistribution which is equal to the second CondGaussianMatrixProductDistribution

In [9]:
class constant(torch.nn.Module):
    
    def __init__(self, vl):
        super().__init__()
        self.vl = torch.nn.Parameter(vl)
    
    def forward(self, x):
        return self.vl

In [10]:
mn_1 = mat_dist_1(x)
std_1 = torch.cat([d.std_f(x) for d in mat_dist_1.dists], dim=1)

In [11]:
mat_dist_2 = CondGaussianDistribution(mn_f=constant(mn_1), std_f=constant(std_1))

In [12]:
mat_dist_0.kl(mat_dist_2, x)

tensor([5.9728, 6.0131, 6.7857, 6.9815, 6.4792, 6.2577, 6.9412, 6.6398, 6.4518,
        5.9142], grad_fn=<SqueezeBackward0>)